In [1]:
import pandas as pd
import numpy as np
import re
import sys
import time
import pickle
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

#from normalization import normalize_corpus,review_to_sentences
from utils import build_feature_matrix
from ast import literal_eval

sys.path.append("/usr/lib/python2.7/dist-packages")

In [2]:
start_script = time.time()
print ''
print "#######################################################################################"
print "Execution Started............"

def timeit(func):
    """
    Simple timing decorator
    """
    def wrapper(*args, **kwargs):
        start  = time.time()
        result = func(*args, **kwargs)
        delta  = time.time() - start
        return result, delta
    return wrapper


def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg



#######################################################################################
Execution Started............


In [3]:
from elasticsearch_dsl import DocType, String, Date, Integer
from elasticsearch_dsl.connections import connections

from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
import requests

index = 'ccthinclient'
doc_type = 'thin_client'

elasticsearch_host = '35.167.139.79:9200'
elastic_url = 'http://35.167.139.79:9200/_bulk?pretty=true'

#elasticsearch_host = '52.24.198.221:8080'
#elastic_url = 'http://52.24.198.221:8080/_bulk?pretty=true'


# Define a default Elasticsearch client
client = connections.create_connection(hosts=[elasticsearch_host])


In [4]:

@timeit
def get_no_of_docs_from_elasticsearch(field):
    body = {}
    if field is 'tags':
        body = {
                "query": {
                    "match":{
                            "taggedbyadmin":"true"
                    }
                }
        }
    else:
         body = {
            "query": {
                "exists": {
                    "field": field
                }
            }
        }
    s = Search.from_dict(body)
    s = s.index(index)
    s = s.doc_type(doc_type)
    s.using(client)
    return s.count()

train_no,secs = get_no_of_docs_from_elasticsearch('tags')
test_no,secs = get_no_of_docs_from_elasticsearch('pathforsearch')
print "Total train documents : ", train_no
print "Total test documents : ", test_no

Total train documents :  4
Total test documents :  330


In [ ]:

@timeit
def read_elasticsearch_data(to_i):
    body = {
                "from": 0,
                "size": to_i,
                "query": {
                    "match":{
                        "taggedbyadmin":"true"
                      }
                    
                }
            }
    s = Search.from_dict(body)
    s = s.index(index)
    s = s.doc_type(doc_type)
    s.using(client)
    return s.execute()
print 'Reading train documents.........'
raw_train, secs = read_elasticsearch_data(train_no)
print 'Time taken to read',  train_no ,'train document is : ', secs 


In [ ]:
def read_elasticsearch_data_test(from_i, to_i, field):
    body = {
                "from": from_i,
                "size": to_i,
                "query": {
                    "exists": {
                        "field": field
                    }
                }
            }
    s = Search.from_dict(body)
    s = s.index(index)
    s = s.doc_type(doc_type)
    s.using(client)
    return s.execute()

start_test_read  = time.time()

print 'Reading test documents.........'
raw_test =[]
single_time_read_count = 100
counter = 0
while test_no > counter * single_time_read_count:
    raw_test.extend(read_elasticsearch_data_test(counter * single_time_read_count, single_time_read_count, "pathforsearch"))
    counter = counter + 1


print 'Time taken to read',  test_no ,'train document is : ', time.time() - start_test_read

In [ ]:


def createDataFrame(rawdata, data_use):
    fileid = []
    text = []
    tags = []
    names = []
    for i in range(len(rawdata)):
        if('file_text' in rawdata[i] and
           'name' in rawdata[i] and
           len(rawdata[i].file_text.strip()) > 0 and
              'file_id' in rawdata[i]):
            
            if ('train' in data_use and 'taggedbyadmin' in rawdata[i] and
                len(rawdata[i].tags) > 0):
                text.append(rawdata[i].file_text)
                fileid.append(rawdata[i].file_id)
                tags.append(rawdata[i].tags)
                names.append(rawdata[i].name)
            elif('test' in data_use and 'taggedbyadmin' not in rawdata[i]):
                text.append(rawdata[i].file_text)
                fileid.append(rawdata[i].file_id)
                tags.append([])
                names.append(rawdata[i].name)
    return pd.DataFrame(
        {'index': fileid,
         'text': text,
         'tags': tags,
         'name': names
        })
     

train = createDataFrame(raw_train, 'train')
print 'size of train documents %d rows and %d columns' %train.shape
test = createDataFrame(raw_test, 'test')
print 'size of test documents %d rows and %d columns' %test.shape

In [ ]:
import nltk
import re
import string

from nltk.corpus import wordnet as wn
from contractions import CONTRACTION_MAP
from nltk.stem import WordNetLemmatizer
from HTMLParser import HTMLParser
import unicodedata

wnl = WordNetLemmatizer()
html_parser = HTMLParser()

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get',
                                 'tell', 'listen', 'one', 'two', 'three',
                                 'four', 'five', 'six', 'seven', 'eight',
                                 'nine', 'zero', 'join', 'find', 'make',
                                 'say', 'ask', 'tell', 'see', 'try', 'back',
                                 'also', '', None, ' ']


def expand_contractions(text, contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text
    

def word_tokenize(text):
    tokens = nltk.word_tokenize(text) 
    tokens = [token.strip() for token in tokens]
    return tokens

def remove_special_characters(text):
    tokens = text
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
    return filtered_tokens
    
    
def remove_stopwords(text):
    tokens = text
    filtered_tokens = [token for token in tokens if token.strip() not in stopword_list]
    return filtered_tokens


# Annotate text tokens with POS tags
def pos_tag_text(words):
    
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None
        
        
    def pos_tagging(words):
        return nltk.pos_tag(words)
    
    tagged_words = pos_tagging(words)
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag))
                         for word, pos_tag in
                         tagged_words]
    return tagged_lower_text
    


def lemmatize_text(words):
    pos_tagged_words = pos_tag_text(words)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
                         else word                     
                         for word, pos_tag in pos_tagged_words]
    return lemmatized_tokens

def parse_hippa(text):
    hippa_pattern = '[[^a-zA-Z0-9]*/*[[^a-zA-Z0-9., -_]*/*[^a-zA-Z0-9., -_]*]*~]*'    
    if re.match(hippa_pattern, text)    :
        text = re.sub('\*','\* ', text)
        text = re.sub('~',' ~ ', text)
        return True, text
 
    return False, text 



def normalize_corpus(line, lemmatize=True):
    
    line = html_parser.unescape(line)
    line = expand_contractions(line, CONTRACTION_MAP)
    
    ishippa, line = parse_hippa(line)
    
    words = word_tokenize(line)
    
    if not ishippa:
        words = remove_special_characters(words)
    
    words = remove_stopwords(words)
    
    if lemmatize:
        words = lemmatize_text(words)
    return ' '.join(words)



### Normalize train Data

In [ ]:
print 'train data preprocessing started...........'
start_train_preprocess  = time.time()
norm_corpus = train.text.apply(normalize_corpus)
print 'Model data preprocessing is finished in : ', time.time() - start_train_preprocess

In [ ]:
start_model = time.time()
print 'model building started...........'

smooth_idf = True
use_idf = True
min_df = 0.1
max_df = 0.8
sublinear_tf = True
threshold_cosine =0.3

if len(norm_corpus) == 1:
    smooth_idf = True
    use_idf = True
    min_df = 1
    max_df = 1.0
    sublinear_tf=True
    threshold_cosine=0.4

tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_corpus,
                                                feature_type='tfidf',
                                                ngram_range=(1, 3), 
                                                min_df=min_df, max_df=max_df, sublinear_tf=sublinear_tf,
                                                use_idf=use_idf, smooth_idf=smooth_idf)
print('Model building finished in : ', time.time() - start_model)

### Normalize test Data

In [ ]:
start_test_preprocess  = time.time()
# normalize and extract features from the query corpus
print 'Test data preprocessing started.........'
norm_query_docs =  test.text.apply(normalize_corpus) #(test.text, lemmatize=True)
#norm_query_docs = [' '.join(doc) for doc in norm_query_docs]
print 'Test data preprocessing is finished in : ', time.time() - start_test_preprocess

In [ ]:
start_vectorise  = time.time()
print 'Test data vectorzing started.......'
query_docs_tfidf = tfidf_vectorizer.transform(norm_query_docs)
print 'Test data vectorizing is finished in : ', time.time() - start_vectorise

### Cosine Similarity

In [ ]:

from collections import Counter
from itertools import groupby
from itertools import chain


class TagScore:
    def __init__(self, tag, sim_score):
        self.tag = tag
        self.sim_score = sim_score


def get_top_tag(top5):
    toptag_dir = {}
    if len(top5) > 0:
        for row in top5:
            if isinstance(row, TagScore):
                for tag in row.tag:
                    if(tag in toptag_dir and 
                       toptag_dir[str(tag)] > row.sim_score):
                        pass
                    else :
                        toptag_dir[str(tag)] = row.sim_score
    return toptag_dir


def compute_cosine_similarity(doc_features, corpus_features,
                              top_n=3):
    # get document vectors
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()
    # compute similarities
    similarity = np.dot(doc_features, 
                        corpus_features.T)
    # get docs with highest similarity scores
    top_docs = similarity.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(similarity[index], 3))
                            for index in top_docs]
    return top_docs_with_score


print 'Document Similarity Analysis using Cosine Similarity'
print('index', '||', 'tagscore', '||', 'top_tag' , '||', 'actual_tag')
start_cosine = time.time()
tag_pred_cos = []
json_update = []
for index, doc in enumerate(test.text):
    
    doc_tfidf = query_docs_tfidf[index]
    top_similar_docs = compute_cosine_similarity(doc_tfidf,
                                             tfidf_features,
                                             top_n=3)
    top5 = []
    sim_scores = []
    for doc_index, sim_score in top_similar_docs:
        if sim_score > threshold_cosine :
            top5.append(TagScore(train.tags[doc_index], sim_score))
            

    tagsscore_dir = get_top_tag(top5)
    
    tag_pred_cos.append(tagsscore_dir)
    key = test['index'][index]
    tags = str(tagsscore_dir.keys()).replace('\'', '"')
    tagscore = str(tagsscore_dir).replace('\'', '"')
    print(index, '|', tagscore, '|', tags, '|', test.name[index])
    json_update.append('''
    { \"update\": { \"_index\": \"ccthinclient\", \"_type\": \"thin_client\", \"_id\": \"%s\", \"_retry_on_conflict\" : 1} }
    { \"script\":{  \"inline\": \"ctx._source.tagscore=params.tagscore;ctx._source.tags=params.tags\", \"params\":{ \"tagscore\":%s, \"tags\":%s }   } }'''%(key, tagscore, tags))

json_update.append('''\n''')    
print 'Time taken to predict data cosine : ', time.time() - start_cosine
test['pred_tags_cosine'] = tag_pred_cos


In [ ]:
raw_train[0].file_id

### Hellinger Bhattacharya Distance

### Update ElasticSearch with predicted tag using cosine

In [ ]:
def elasticsearch_bulk_update(to_update) :
    start_update  = time.time()
    print 'Updating elasticsearch data with Cosine Distance.......'
    response = requests.post(elastic_url, data=''.join(to_update))
    print'elasticsearch data updated in ', time.time() - start_update, 'sec'

elasticsearch_bulk_update(json_update) 

In [ ]:
print ''
print 'Total time for one execution is: ' , time.time()-start_script
print ''
#except Exception as e:
#        print(e)

# While converting to .py delete below cells

In [ ]:
json_update[37]

In [ ]:
for row in raw_train:
    print row.file_id, row.tags, row.name

### Create CSV of predicted tags

In [ ]:
    
test['pred_tags_h_b'] = tag_pred_h_b
preddf = test[['index', 'pred_tags_cosine', 'pred_tags_h_b']]
preddf.to_csv('predicted_tags.csv')
print('tags are predicted in predicted_tags.csv')

### Update Elastic Search Trials

In [ ]:
import requests
url = 'http://35.167.139.79:9200/_bulk?pretty=true'
data = """{ "update": { "_index": "ccthinclient", "_type": "thin_client", "_id": "d86cb9d277a3f6ae32dcb21251ee15ce0705a955", "_retry_on_conflict" : 1} }
{ "doc" : {"tags" : ["tagsss"]}} 
{ "update": { "_index": "ccthinclient", "_type": "thin_client", "_id": "d86cb9d277a3f6ae32dcb21251ee15ce0705a955", "_retry_on_conflict" : 1} }
{ "doc" : {"tags" : ["tagsss"]}} """

response = requests.post(url, data=data)

#### Update actual Tags

#### Update tags to default value tagsss

In [ ]:
import requests
url = 'http://35.167.139.79:9200/_bulk?pretty=true'
json_update = ''
data = []

for i in range(len(test)):
    key = test['index'][i]

    tagsscore_dir = {"a": 0.5}
    tagscore = str(tagsscore_dir).replace('\'', '"')
    tag = str(tagsscore_dir.keys()).replace('\'', '"')
    data.append('''
    { \"update\": { \"_index\": \"ccthinclient\", \"_type\": \"thin_client\", \"_id\": \"%s\", \"_retry_on_conflict\" : 1} }
    { \"script\":{  \"inline\": \"ctx._source.tagscore=params.tagscore;ctx._source.tags=params.tags\", \"params\":{ \"tagscore\":%s, \"tags\":%s }   } }'''%(key,tagscore,tag))
  
    data.append('''\n''')
response = requests.post(url, data=''.join(data))  

In [ ]:
for i in range(len(test)):
    key = test['index'][i]
    if "5c91404b59c9d4a093ab830e78dbcf566c547563" in key:
        print True

### Read all documents and check tags

In [ ]:
client = connections.create_connection(hosts=[elasticsearch_host])
raw_test = read_elasticsearch_data_test(0, 20, "pathforsearch")
for i in range(len(raw_test)):
    if('file_text' in raw_test[i] and 'name' in raw_test[i] and len(raw_test[i].file_text.strip()) > 0 and 
       'file_id' in raw_test[i]):
            
        print(str(raw_test[i].file_id) + "::::::" + str(raw_test[i].tagscore) + raw_test[i].name)

In [ ]:
len([x for x in tag_pred_cos if 'mobile' in x])

In [ ]:
import re
count = 0
for nm in test.name:
    if re.search('mc[a-z0-9.]*', nm):
        count = count +1
    if re.search('nc[a-z0-9.]*', nm):
        count = count +1
    
print(count)

In [ ]:
len(raw_test)

In [ ]:
curl -XPOST 'localhost:9200/_bulk?pretty' -H 'Content-Type: application/json' -d'
{ "update": { "_index": "ccthinclient", "_type": "thin_client", "_id": "a7e61fe696ad59cd43cbac6a4a8fa8e1313e0698", "_retry_on_conflict" : 1} }
{ "doc" : {"tags" : ["test"]}} 
{ "update": { "_index": "ccthinclient", "_type": "thin_client", "_id": "58a5749e362a476fe18812038951a3c696c576cf", "_retry_on_conflict" : 1} }
{ "doc" : {"tags" : ["test1"]}} 
'

In [ ]:
import requests
url = 'http://35.167.139.79:9200/_bulk?pretty=true'
json_update = ''
data = []

for i in range(len(test)):
    key = test['index'][i]

    tagsscore_dir = {"a": 0.5}
    tagscore = str(tagsscore_dir).replace('\'', '"')
    tag = str(tagsscore_dir.keys()).replace('\'', '"')
    data.append('''
    { \"update\": { \"_index\": \"ccthinclient\", \"_type\": \"thin_client\", \"_id\": \"%s\", \"_retry_on_conflict\" : 1} }
    { \"script\":{  \"inline\": \"ctx._source.tagscore=params.tagscore;ctx._source.tags=params.tags\", \"params\":{ \"tagscore\":%s, \"tags\":%s }   } }'''%(key,tagscore,tag))
  
    data.append('''\n''')
response = requests.post(url, data=''.join(data))  

In [ ]:
curl -XGET "http://35.167.139.79:9200/_analyze?tokenizer=standard&char_filters=html_strip&analyzer=english" -d'
{
  "text" : "ISA* 00*           * 00*           * ZZ* EMEDNYBAT      * ZZ* ETIN           * 100101* 1000* ^* 00501* 006000600* 0* T* :~GS* HP* EMEDNYBAT* ETIN* 20100101* 1050* 6000600* X* 005010X221A1~ST* 835* 1740~BPR* H* 0* C* NON* * * * * * * * * * * * 20100101~TRN* 1* 10100000000* 1000000000~REF* EV* ETIN~DTM* 405* 20100101~N1* PR* NYSDOH~N3* OFFICE OF HEALTH INSURANCE PROGRAMS* CORNING TOWER, EMPIRE STATE PLAZA~N4* ALBANY* NY* 122370080~PER* BL* PROVIDER SERVICES* TE* 8003439000* UR* www.emedny.org~N1* PE* MAJOR MEDICAL PROVIDER* XX* 9999999995~REF* TJ* 000000000~LX* 1~CLP* PATIENT ACCOUNT NUMBER* 2* 34* 0* * MC* 1000220000000030* 11~NM1* QC* 1* SUBMITTED LAST* SUBMITTED FIRST* * * * MI* LL88888L~NM1* 74* 1* CORRECTED LAST* CORRECTED FIRST~REF* EA* PATIENT ACCOUNT NUMBER~DTM* 232* 20100101~DTM* 233* 20100101~SVC* HC:V2020* 12* 0* * 0~DTM* 472* 20100101~CAS* CO* 29* 12~SVC* HC:V2103* 22* 0* * 0~DTM* 472* 20100101~CAS* CO* 29* 22~SE* 25* 1740~GE* 1* 6000600~IEA* 1* 006000600~"
}'


In [ ]:
curl -XGET "http://35.167.139.79:9200/_analyze?analyzer=english&tokenizer=standard" -d'
{
  "text" : "This is a test, which you have to pass man."
}'

In [ ]:
xml = '''<web-app xmlns="http://java.sun.com/xml/ns/j2ee"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://java.sun.com/xml/ns/j2ee http://java.sun.com/xml/ns/j2ee/web-app_2_4.xsd"
    version="2.4">

    <display-name>HelloWorld Application</display-name>
    <description>
        This is a simple web application with a source code organization
        based on the recommendations of the Application Developer's Guide.
    </description>

    <servlet>
        <servlet-name>HelloServlet</servlet-name>
        <servlet-class>examples.Hello</servlet-class>
    </servlet>

    <servlet-mapping>
        <servlet-name>HelloServlet</servlet-name>
        <url-pattern>/hello</url-pattern>
    </servlet-mapping>

</web-app>   '''

In [ ]:
string = '''{
  "tokenizer": "<([^<>]+)>([^<>]*)<(\/[^<>]+)>",
  "text": %s
}'''%(xml)
'''
curl -XPOST 'http://35.167.139.79:9200/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "tokenizer": "<([^<>]+)>([^<>]*)<(\/[^<>]+)>",
  "text": "<servlet-mapping> <servlet-name>HelloServlet</servlet-name> <url-pattern>/hello</url-pattern> </servlet-mapping>"
}
'

'''

In [ ]:
url = 'http://35.167.139.79:9200/_analyze?pretty=true'
response = requests.post(url, data=string)  

In [ ]:
response.text

In [ ]:
curl -XPUT 'http://35.167.139.79:9200/xml_index?pretty' -H 'Content-Type: application/json' -d'
{
  "settings": {
    "analysis": {
      "analyzer": {
        "xml_analyzer": {
          "tokenizer": "xml_tokenizer"
        }
      },
      "tokenizer": {
        "xml_tokenizer": {
          "type": "pattern",
          "pattern": "<([^<>]+)>([^<>]*)<(\/[^<>]+)>"
        }
      }
    }
  }
}
'

curl -XPOST 'http://35.167.139.79:9200/xml_index/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "analyzer": "xml_analyzer",
  "text": "<servlet-mapping> <servlet-name>HelloServlet</servlet-name> <url-pattern>/hello</url-pattern> </servlet-mapping>"
}
'
